# **AI-Driven Cable Design Validation System**
*LangGraph Multi-Agent Workflow*

In [162]:
!pip install -U langchain==0.2.7 langgraph langchain-google-genai==1.0.3 pydantic

  Using cached langchain-0.2.7-py3-none-any.whl.metadata (6.9 kB)
  Using cached langchain_google_genai-1.0.3-py3-none-any.whl.metadata (3.8 kB)
  Using cached langchain_core-0.2.43-py3-none-any.whl.metadata (6.2 kB)
  Using cached langchain_text_splitters-0.2.4-py3-none-any.whl.metadata (2.3 kB)
  Using cached langsmith-0.1.147-py3-none-any.whl.metadata (14 kB)
  Using cached numpy-1.26.4-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
  Using cached tenacity-8.5.0-py3-none-any.whl.metadata (1.2 kB)
  Using cached google_generativeai-0.5.4-py3-none-any.whl.metadata (3.9 kB)
INFO: pip is looking at multiple versions of langchain-google-genai to determine which version is compatible with other requirements. This could take a while.
ERROR: Cannot install langchain-google-genai==1.0.3 and langchain==0.2.7 because these package versions have conflicting dependencies.

The conflict is caused by:
    langchain 0.2.7 depends on langchain-core<0.3.0 and >=0.2.12
   

In [ ]:
import os
import google.generativeai as genai
os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY", "")
genai.configure(api_key=os.environ["GOOGLE_API_KEY"])

## IMPORTS

In [164]:
from typing import TypedDict, Literal, Optional, List, Dict, Any
from langgraph.graph import StateGraph, END
from langchain_google_genai import ChatGoogleGenerativeAI
import json
import os
import re

## CONFIGURATION

In [165]:
for model in genai.list_models():
    print(f"{model.name} supports: {model.supported_generation_methods}")

models/embedding-gecko-001 supports: ['embedText', 'countTextTokens']
models/gemini-2.5-flash supports: ['generateContent', 'countTokens', 'createCachedContent', 'batchGenerateContent']
models/gemini-2.5-pro supports: ['generateContent', 'countTokens', 'createCachedContent', 'batchGenerateContent']
models/gemini-2.0-flash-exp supports: ['generateContent', 'countTokens', 'bidiGenerateContent']
models/gemini-2.0-flash supports: ['generateContent', 'countTokens', 'createCachedContent', 'batchGenerateContent']
models/gemini-2.0-flash-001 supports: ['generateContent', 'countTokens', 'createCachedContent', 'batchGenerateContent']
models/gemini-2.0-flash-exp-image-generation supports: ['generateContent', 'countTokens', 'bidiGenerateContent']
models/gemini-2.0-flash-lite-001 supports: ['generateContent', 'countTokens', 'createCachedContent', 'batchGenerateContent']
models/gemini-2.0-flash-lite supports: ['generateContent', 'countTokens', 'createCachedContent', 'batchGenerateContent']
models/ge

In [166]:
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0.1
)

## MOCK DATABASE

In [167]:
DESIGN_DATABASE = {
    "DESIGN-001": {
        "standard": "IEC 60502-1",
        "voltage": "0.6/1 kV",
        "conductor_material": "Cu",
        "conductor_class": "Class 2",
        "csa": 10,
        "insulation_material": "PVC",
        "insulation_thickness": 1.0
    },
    "DESIGN-002": {
        "standard": "IEC 60502-1",
        "voltage": "0.6/1 kV",
        "conductor_material": "Cu",
        "conductor_class": None,  # Missing attribute
        "csa": 16,
        "insulation_material": "PVC",
        "insulation_thickness": None  # Missing attribute
    }
}

In [168]:
# Required attributes for validation
REQUIRED_ATTRIBUTES = [
    "standard",
    "voltage",
    "conductor_material",
    "conductor_class",
    "csa",
    "insulation_material",
    "insulation_thickness"
]

## STATE DEFINITION

In [169]:
class CableValidationState(TypedDict):
    user_input: str
    route: Optional[str]
    design_id: Optional[str]
    attributes: Dict[str, Any]
    missing_attributes: List[str]
    validation: Optional[List[Dict[str, Any]]]
    reasoning: Optional[str]
    confidence: Optional[float]
    conversation_history: List[str]
    skip_missing_prompts: bool
    initial_validation_done: bool
    hitl_mode: bool

## NODE 1: SUPERVISOR AGENT (LLM-BASED ROUTER)

In [170]:
def supervisor_agent(state: CableValidationState) -> CableValidationState:
    """
    FIXED: Enhanced prompt with clear examples and pattern matching logic
    """
    user_input = state["user_input"]

    prompt = f"""You are a routing supervisor for a cable design validation system.

Classify this input into ONE route:

**FETCH_DESIGN** - Input contains design ID pattern (DESIGN-XXX)
Examples: "Validate DESIGN-001", "Check design DESIGN-002"

**EXTRACT_FROM_TEXT** - Input contains cable technical specifications
Examples: "IEC 60502-1, 10mm² Cu", "0.6/1kV cable with PVC insulation"

**IGNORE** - Input is unrelated to cable design
Examples: "What's the weather?", "Tell me a joke"

Input: "{user_input}"

Respond ONLY with JSON (no markdown):
{{"route":"FETCH_DESIGN"}} or {{"route":"EXTRACT_FROM_TEXT"}} or {{"route":"IGNORE"}}"""

    response = llm.invoke(prompt)

    try:
        content = response.content.strip()
        content = re.sub(r'```json\s*|\s*```', '', content)
        match = re.search(r'\{[^}]+\}', content)
        if match:
            result = json.loads(match.group())
            route = result.get("route", "IGNORE")
        else:
            route = "IGNORE"
    except:
        input_lower = user_input.lower()
        if "design-" in input_lower:
            route = "FETCH_DESIGN"
        elif any(kw in input_lower for kw in ["iec", "kv", "copper", "cu", "cable", "insulation"]):
            route = "EXTRACT_FROM_TEXT"
        else:
            route = "IGNORE"

    print(f"\n🎯 SUPERVISOR DECISION: {route}")
    state["route"] = route
    return state

## NODE 2: FETCH DESIGN NODE

In [171]:
def fetch_design_node(state: CableValidationState) -> CableValidationState:
    """Fetch design from database"""
    user_input = state["user_input"]

    pattern = r'DESIGN-\d+'
    match = re.search(pattern, user_input, re.IGNORECASE)

    if match:
        design_id = match.group().upper()
    else:
        prompt = f"""Extract design ID from: "{user_input}"
Return JSON: {{"design_id": "DESIGN-XXX"}} or {{"design_id": null}}"""
        response = llm.invoke(prompt)
        try:
            content = re.sub(r'```json\s*|\s*```', '', response.content.strip())
            match = re.search(r'\{[^}]+\}', content)
            result = json.loads(match.group()) if match else {}
            design_id = result.get("design_id")
        except:
            design_id = None

    if design_id and design_id in DESIGN_DATABASE:
        attributes = DESIGN_DATABASE[design_id].copy()
        print(f"\n📦 FETCHED DESIGN: {design_id}")
        state["design_id"] = design_id
        state["attributes"] = attributes
    else:
        print(f"\n❌ DESIGN NOT FOUND: {design_id}")
        state["attributes"] = {}

    return state

## NODE 3: EXTRACT FROM TEXT NODE

In [172]:
def extract_from_text_node(state: CableValidationState) -> CableValidationState:
    """Extract cable specifications from text"""
    user_input = state["user_input"]

    prompt = f"""Extract cable specifications from text. Return ONLY values explicitly stated.

Input: "{user_input}"

Rules:
- Set field to null if not mentioned
- Do NOT infer missing values
- Extract exact values only

Output JSON format:
{{
  "standard": string or null,
  "voltage": string or null,
  "conductor_material": string or null,
  "conductor_class": string or null,
  "csa": number or null,
  "insulation_material": string or null,
  "insulation_thickness": number or null
}}

Examples:
"10 sqmm copper cable" → {{"csa": 10, "conductor_material": "Cu", others: null}}
"IEC 60502-1, 0.6/1kV, Cu Class 2, 16mm², PVC 1.0mm" → all fields filled

Extract now (JSON only):"""

    response = llm.invoke(prompt)

    try:
        content = response.content.strip()
        content = re.sub(r'```json\s*|\s*```', '', content)
        start = content.find('{')
        end = content.rfind('}') + 1
        if start != -1 and end > start:
            attributes = json.loads(content[start:end])
            print(f"\n🔍 EXTRACTED ATTRIBUTES:")
            for key, value in attributes.items():
                print(f"   {'✓' if value else '✗'} {key}: {value}")
            state["attributes"] = attributes
        else:
            raise ValueError("No JSON found")
    except Exception as e:
        print(f"\n❌ EXTRACTION FAILED: {e}")
        state["attributes"] = {}

    return state

## NODE 4: COMPLETENESS CHECKER (DETERMINISTIC)

In [173]:
def check_missing_attributes(state: CableValidationState) -> CableValidationState:
    """Check for missing attributes"""
    attributes = state.get("attributes", {})
    missing = [attr for attr in REQUIRED_ATTRIBUTES
               if attributes.get(attr) is None or attributes.get(attr) == ""]

    state["missing_attributes"] = missing

    if missing:
        print(f"\n⚠️  MISSING ATTRIBUTES: {missing}")
        if state.get("hitl_mode", False):
            print("   → Will proceed to HITL interaction after initial validation")
        else:
            print("   → Will validate with WARN status for missing fields")
    else:
        print(f"\n✅ ALL ATTRIBUTES PRESENT")

    return state

## NODE 6: VALIDATION AGENT (LLM REASONING)

In [174]:
def validation_agent(state: CableValidationState) -> CableValidationState:
    """
    FIXED VALIDATION AGENT - Implements correct WARN logic and confidence calibration
    """
    attributes = state["attributes"]
    missing = state.get("missing_attributes", [])
    is_initial = not state.get("initial_validation_done", False)

    prompt = f"""You are an expert cable design validation engineer.

**VALIDATION INPUT:**
{json.dumps(attributes, indent=2)}

**MISSING FIELDS:** {missing if missing else "None"}
**VALIDATION TYPE:** {"Initial validation with WARN for missing fields" if is_initial else "Re-validation after HITL interaction"}

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
CRITICAL INSTRUCTIONS - READ CAREFULLY
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

**1. STATUS CLASSIFICATION RULES (MANDATORY)**

**PASS** - Use when:
✓ Value matches IEC nominal exactly
✓ Value exceeds nominal (e.g., 1.1mm when 1.0mm required)
✓ All requirements fully satisfied
✓ No ambiguity or missing context

**WARN** - Use when:
⚠ Value is 85-99% of nominal (borderline acceptable)
  Example: 0.9mm when nominal is 1.0mm → WARN (90% of nominal)
  Example: 0.85mm when nominal is 1.0mm → WARN (85% of nominal)
⚠ Field is null/missing (cannot validate without data)
⚠ Value is 101-110% of nominal (acceptable but non-standard)
⚠ Standard is missing (validation basis unclear)
⚠ Context insufficient for definitive judgment

**FAIL** - Use when:
✗ Value is <85% of nominal (safety risk)
  Example: 0.5mm when nominal is 1.0mm → FAIL (50% of nominal)
  Example: 0.8mm when nominal is 1.0mm → FAIL (80% of nominal)
✗ Value does not exist in IEC tables
✗ Prohibited combination
✗ Clear safety violation

**TOLERANCE PHILOSOPHY:**
- Nominal values in IEC standards represent design targets
- Manufacturing tolerances typically allow ±10% variation
- 85-99% of nominal = borderline but potentially acceptable (WARN)
- <85% of nominal = unacceptable safety risk (FAIL)
- >110% of nominal = over-designed, acceptable but atypical (WARN or PASS)

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

**2. CONFIDENCE SCORING RULES (MANDATORY)**

Base confidence calculation:
- Start at 1.0 (perfect confidence)
- Subtract for each uncertainty factor:
  * Missing critical field: -0.15 per field
  * WARN status: -0.10 per WARN
  * Missing standard: -0.25
  * FAIL status: -0.05 (decisive but problem exists)
  * Ambiguous input: -0.20

Target ranges:
- All PASS, no missing data: 0.90-1.0
- 1-2 WARNs: 0.70-0.85
- 3+ WARNs or missing standard: 0.40-0.65
- Mix of WARN+FAIL: 0.50-0.70
- Multiple FAILs: 0.60-0.80 (decisive but multiple issues)

**DO NOT report 0.98-1.0 confidence if there are WARNs or missing fields.**

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

**3. IEC STANDARDS REFERENCE FRAMEWORK**

**IEC 60228** - Conductors of insulated cables:
- Validates: conductor_material, conductor_class, csa
- Table 1: Nominal CSA values (1.5, 2.5, 4, 6, 10, 16, 25, 35, 50, 70, 95...)
- Cu (Copper) and Al (Aluminum) both valid
- Class 1 = solid, Class 2 = stranded

**IEC 60502-1** - Power cables 0.6/1 kV to 18/30 kV:
- Validates: voltage, insulation_material, insulation_thickness
- Table 1 (PVC): CSA-specific nominal insulation thickness
- Table 2 (XLPE/EPR): Different thickness requirements
- Voltage range: 0.6/1 kV to 18/30 kV

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

**4. VALIDATION EXAMPLES (STUDY THESE)**

**Example 1: Borderline insulation (WARN not FAIL)**
Input: insulation_thickness = 0.9mm, nominal = 1.0mm
Status: WARN
Reasoning: "0.9mm represents 90% of the nominal 1.0mm specified in IEC 60502-1 Table 1 for 10mm² PVC conductors. While below nominal, this falls within potential manufacturing tolerance range (±10%) and may be acceptable. Engineering review recommended."
Confidence impact: -0.10

**Example 2: Missing standard (WARN)**
Input: standard = null
Status: WARN
Reasoning: "No IEC standard specified. Unable to determine applicable validation criteria. Assuming IEC 60502-1 based on voltage rating, but validation basis is uncertain."
Confidence impact: -0.25

**Example 3: Critically low insulation (FAIL)**
Input: insulation_thickness = 0.5mm, nominal = 1.0mm
Status: FAIL
Reasoning: "0.5mm represents only 50% of nominal 1.0mm per IEC 60502-1 Table 1. This is critically below minimum requirements and poses significant safety risk for 0.6/1kV operation."
Confidence impact: -0.05

**Example 4: Missing conductor class (WARN)**
Input: conductor_class = null
Status: WARN
Reasoning: "Conductor class not specified. IEC 60228 requires classification as Class 1 (solid) or Class 2 (stranded). Cannot verify conductor construction compliance."
Confidence impact: -0.15

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

**5. OUTPUT FORMAT (STRICT)**

{{
  "validation": [
    {{
      "field": "standard",
      "status": "PASS|WARN|FAIL",
      "expected": "IEC 60502-1",
      "comment": "Detailed explanation with specific IEC references"
    }},
    {{
      "field": "voltage",
      "status": "PASS|WARN|FAIL",
      "expected": "0.6/1 kV to 18/30 kV range (IEC 60502-1)",
      "comment": "..."
    }},
    {{
      "field": "conductor_material",
      "status": "PASS|WARN|FAIL",
      "expected": "Cu or Al per IEC 60228",
      "comment": "IEC 60228 permits both copper and aluminum conductors"
    }},
    {{
      "field": "conductor_class",
      "status": "PASS|WARN|FAIL",
      "expected": "Class 1 or Class 2 per IEC 60228",
      "comment": "..."
    }},
    {{
      "field": "csa",
      "status": "PASS|WARN|FAIL",
      "expected": "Nominal value from IEC 60228 Table 1",
      "comment": "Must match standard sizes: 1.5, 2.5, 4, 6, 10, 16, 25, 35..."
    }},
    {{
      "field": "insulation_material",
      "status": "PASS|WARN|FAIL",
      "expected": "PVC, XLPE, or EPR per IEC 60502-1",
      "comment": "..."
    }},
    {{
      "field": "insulation_thickness",
      "status": "PASS|WARN|FAIL",
      "expected": "Nominal from IEC 60502-1 Table 1/2",
      "comment": "For 10mm² PVC at 0.6/1kV, nominal is 1.0mm. Actual value assessment with tolerance consideration."
    }}
  ],
  "reasoning": "Overall assessment explaining compliance level, cumulative uncertainty from missing fields, and engineering judgment",
  "confidence": 0.75
}}

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

**6. MANDATORY REQUIREMENTS**

✓ Validate ALL 7 fields (even if null)
✓ Apply tolerance logic: 85-99% nominal = WARN
✓ Missing field = WARN (not interactive prompt)
✓ Calculate confidence using deduction rules
✓ Reference both IEC 60228 and IEC 60502-1 where applicable
✓ Cite specific table numbers
✓ Explain WARN decisions clearly (not just "needs review")

**NOW VALIDATE THE DESIGN ABOVE**
Return ONLY valid JSON (no markdown, no preamble):"""

    response = llm.invoke(prompt)

    try:
        content = response.content.strip()
        content = re.sub(r'```json\s*|\s*```', '', content)
        start = content.find('{')
        end = content.rfind('}') + 1

        if start != -1 and end > start:
            result = json.loads(content[start:end])

            validation = result.get("validation", [])
            reasoning = result.get("reasoning", "")
            confidence = result.get("confidence", 0.0)

            # Post-process: Ensure confidence aligns with status
            warn_count = sum(1 for v in validation if v.get("status") == "WARN")
            fail_count = sum(1 for v in validation if v.get("status") == "FAIL")

            # Recalibrate if AI still reports inflated confidence
            if warn_count > 0 or fail_count > 0 or missing:
                max_confidence = 1.0 - (warn_count * 0.10) - (len(missing) * 0.15) - (fail_count * 0.05)
                if confidence > max_confidence:
                    print(f"\n⚙️  Recalibrating confidence: {confidence:.2f} → {max_confidence:.2f}")
                    confidence = max(0.3, max_confidence)

            state["validation"] = validation
            state["reasoning"] = reasoning
            state["confidence"] = confidence
            state["initial_validation_done"] = True  # Mark initial validation as done

            print(f"\n🔬 VALIDATION COMPLETE")
            print(f"   Confidence: {confidence:.2f}")
            print(f"   WARN count: {warn_count}, FAIL count: {fail_count}, Missing: {len(missing)}")
        else:
            raise ValueError("No valid JSON found")

    except Exception as e:
        print(f"\n❌ VALIDATION FAILED: {e}")
        state["validation"] = []
        state["reasoning"] = "Validation could not be completed"
        state["confidence"] = 0.0

    return state

## ROUTING FUNCTIONS

In [175]:
def route_after_supervisor(state: CableValidationState) -> str:
    route = state.get("route", "IGNORE")
    return {"IGNORE": "end", "FETCH_DESIGN": "fetch_design", "EXTRACT_FROM_TEXT": "extract_from_text"}.get(route, "end")

In [176]:
def route_after_check(state: CableValidationState) -> str:
    """Route after checking missing attributes"""
    # Always validate first to get WARN status
    return "validate"

In [177]:
def route_after_validation(state: CableValidationState) -> str:
    """Route after validation - decide if HITL needed"""
    missing = state.get("missing_attributes", [])
    hitl_mode = state.get("hitl_mode", False)
    initial_done = state.get("initial_validation_done", False)

    # If HITL mode enabled, missing fields exist, and this is the first validation
    if hitl_mode and missing and initial_done:
        # Check if there are WARN statuses due to missing fields
        validation = state.get("validation", [])
        has_missing_warns = any(
            v['status'] == 'WARN' and v['field'] in missing
            for v in validation
        )
        if has_missing_warns:
            return "hitl_prompt"

    return "end"

In [178]:
def hitl_prompt_node(state: CableValidationState) -> CableValidationState:
    """Human-in-the-Loop: Prompt for missing attributes"""
    missing = state.get("missing_attributes", [])
    validation = state.get("validation", [])

    print("\n" + "="*80)
    print("🤝 HUMAN-IN-THE-LOOP INTERACTION")
    print("="*80)
    print("\n⚠️  The following fields have WARN status due to missing data:")

    # Show which fields need input
    for field in missing:
        warn_item = next((v for v in validation if v['field'] == field), None)
        if warn_item and warn_item['status'] == 'WARN':
            print(f"\n   • {field}")
            print(f"     Status: {warn_item['status']}")
            print(f"     Expected: {warn_item['expected']}")
            print(f"     Reason: {warn_item['comment'][:100]}...")

    print("\n" + "-"*80)
    print("Would you like to provide the missing information? (yes/no)")
    user_choice = input(">>> ").strip().lower()

    if user_choice in ['yes', 'y']:
        state["missing_attributes"] = missing  # Keep missing list for prompting
        return state
    else:
        print("\n✓ Validation complete with current data")
        state["missing_attributes"] = []  # Clear to end workflow
        return state

In [179]:
def ask_missing_attribute(state: CableValidationState) -> CableValidationState:
    """Prompt user for a single missing attribute"""
    missing = state.get("missing_attributes", [])

    if not missing:
        return state

    attr = missing[0]

    attr_display = {
        "standard": "IEC standard (e.g., IEC 60502-1)",
        "voltage": "voltage rating (e.g., 0.6/1 kV)",
        "conductor_material": "conductor material (Cu or Al)",
        "conductor_class": "conductor class (Class 1 or Class 2)",
        "csa": "cross-sectional area in mm² (e.g., 10)",
        "insulation_material": "insulation material (PVC, XLPE, or EPR)",
        "insulation_thickness": "insulation thickness in mm (e.g., 1.0)"
    }

    print(f"\n❓ Please provide the {attr_display.get(attr, attr)}:")
    user_response = input(">>> ").strip()

    if "conversation_history" not in state:
        state["conversation_history"] = []

    state["conversation_history"].append(f"Q: {attr} | A: {user_response}")

    # Parse the user response
    attributes = state.get("attributes", {})
    value = parse_single_attribute_with_llm(llm, attr, user_response)

    if value is not None:
        attributes[attr] = value
        print(f"   ✓ Set {attr} = {value}")

        # Remove from missing list
        missing.remove(attr)
        state["missing_attributes"] = missing
    else:
        print(f"   ✗ Could not extract {attr}. Keeping as missing.")
        # Still remove from list to avoid infinite loop
        missing.remove(attr)
        state["missing_attributes"] = missing

    state["attributes"] = attributes
    return state

In [180]:
def parse_single_attribute_with_llm(
    llm: ChatGoogleGenerativeAI,
    attribute_name: str,
    user_response: str
) -> Any:
    """Extract single attribute value from user input"""
    prompt = f"""Extract ONLY the value for "{attribute_name}" from this user input.

User input: "{user_response}"

Rules:
- Extract ONLY the {attribute_name} value
- Ignore everything else
- Return null if not found or unclear
- For numbers, return as number type
- For text, return as string

Response format (JSON only):
{{"value": <extracted_value>}}

Examples:
Input: "IEC 60502-1" → {{"value": "IEC 60502-1"}}
Input: "10" → {{"value": 10}}
Input: "Class 2" → {{"value": "Class 2"}}"""

    response = llm.invoke(prompt)

    try:
        content = response.content.strip()
        content = re.sub(r'```json\s*|\s*```', '', content)
        match = re.search(r'\{[^}]+\}', content)
        if match:
            parsed = json.loads(match.group())
            return parsed.get("value")
    except Exception:
        pass

    return None

In [181]:
def route_after_hitl_prompt(state: CableValidationState) -> str:
    """Route after HITL prompt - decide if collecting attributes"""
    missing = state.get("missing_attributes", [])
    return "ask_missing" if missing else "revalidate"

In [182]:
def route_after_ask(state: CableValidationState) -> str:
    """Route after asking for attribute"""
    missing = state.get("missing_attributes", [])
    return "ask_missing" if missing else "revalidate"

## GRAPH CONSTRUCTION

In [183]:
def create_validation_graph():
    """Create the validation workflow graph with HITL support"""
    workflow = StateGraph(CableValidationState)

    # Add all nodes
    workflow.add_node("supervisor", supervisor_agent)
    workflow.add_node("fetch_design", fetch_design_node)
    workflow.add_node("extract_from_text", extract_from_text_node)
    workflow.add_node("check_missing", check_missing_attributes)
    workflow.add_node("validate", validation_agent)
    workflow.add_node("hitl_prompt", hitl_prompt_node)
    workflow.add_node("ask_missing", ask_missing_attribute)
    workflow.add_node("revalidate", validation_agent)

    # Entry point
    workflow.set_entry_point("supervisor")

    # Supervisor routing
    workflow.add_conditional_edges("supervisor", route_after_supervisor,
        {"end": END, "fetch_design": "fetch_design", "extract_from_text": "extract_from_text"})

    # Data collection routes
    workflow.add_edge("fetch_design", "check_missing")
    workflow.add_edge("extract_from_text", "check_missing")

    # Initial validation (always happens)
    workflow.add_edge("check_missing", "validate")

    # After validation, check if HITL needed
    workflow.add_conditional_edges("validate", route_after_validation,
        {"end": END, "hitl_prompt": "hitl_prompt"})

    # HITL workflow
    workflow.add_conditional_edges("hitl_prompt", route_after_hitl_prompt,
        {"ask_missing": "ask_missing", "revalidate": "revalidate"})

    workflow.add_conditional_edges("ask_missing", route_after_ask,
        {"ask_missing": "ask_missing", "revalidate": "revalidate"})

    # After revalidation, end
    workflow.add_edge("revalidate", END)

    return workflow.compile()

## EXECUTION & TEST CASES

In [184]:
def run_validation(user_input: str, verbose: bool = True, hitl_mode: bool = False):
    """
    Run cable design validation

    Args:
        user_input: Cable design specification (text or design ID)
        verbose: Print detailed output
        hitl_mode: Enable Human-in-the-Loop for missing attributes
    """
    if verbose:
        print("=" * 80)
        print("AI-DRIVEN CABLE DESIGN VALIDATION SYSTEM")
        print("=" * 80)
        print(f"Input: {user_input}")
        print(f"Mode: {'HITL (Interactive)' if hitl_mode else 'Non-interactive'}\n")

    initial_state = {
        "user_input": user_input,
        "route": None,
        "design_id": None,
        "attributes": {},
        "missing_attributes": [],
        "validation": None,
        "reasoning": None,
        "confidence": None,
        "conversation_history": [],
        "skip_missing_prompts": not hitl_mode,
        "initial_validation_done": False,
        "hitl_mode": hitl_mode
    }

    app = create_validation_graph()
    final_state = app.invoke(initial_state)

    if verbose:
        print("\n" + "=" * 80)
        print("FINAL VALIDATION RESULTS")
        print("=" * 80)

        if final_state.get("validation"):
            for item in final_state["validation"]:
                status_symbol = {"PASS": "✅", "WARN": "⚠️", "FAIL": "❌"}.get(item["status"], "❓")
                print(f"\n{status_symbol} {item['field']}: {item['status']}")
                print(f"   Expected: {item['expected']}")
                print(f"   Comment: {item['comment']}")

            print(f"\n📊 Overall Confidence: {final_state['confidence']:.2f}")
            print(f"\n💭 Reasoning: {final_state['reasoning']}")

            if final_state.get("conversation_history"):
                print(f"\n🤝 HITL Interactions: {len(final_state['conversation_history'])}")
                for interaction in final_state['conversation_history']:
                    print(f"   • {interaction}")
        else:
            print(f"No validation performed (route: {final_state.get('route')})")

        print("\n" + "=" * 80)

    return final_state

## EXAMPLE TEST CASES

In [185]:
if __name__ == "__main__":
    print("""
╔══════════════════════════════════════════════════════════════════════╗
║  AI-DRIVEN CABLE DESIGN VALIDATION - WITH HITL SUPPORT               ║
║  Fixed: WARN logic, confidence calibration, HITL for missing data    ║
╚══════════════════════════════════════════════════════════════════════╝
""")

    print("\n" + "="*80)
    print("MODE 1: NON-INTERACTIVE VALIDATION (Automated)")
    print("="*80)

    print("\n🧪 TEST 1: Complete design (no HITL needed)")
    run_validation(
        "Validate IEC 60502-1 cable, 10 sqmm Cu Class 2, PVC insulation 1.0 mm, 0.6/1 kV",
        hitl_mode=False
    )

    print("\n\n🧪 TEST 2: Incomplete design (WARN status, no prompts)")
    run_validation(
        "10 sqmm copper cable with PVC insulation",
        hitl_mode=False
    )

    print("\n\n" + "="*80)
    print("MODE 2: HUMAN-IN-THE-LOOP VALIDATION (Interactive)")
    print("="*80)
    print("\n🧪 TEST 3: Incomplete design with HITL enabled")
    print("Expected: Initial validation with WARNs, then prompt for missing data\n")

    # Uncomment to test interactive mode
    run_validation(
        "10 sqmm copper cable with PVC insulation",
        hitl_mode=True
    )


╔══════════════════════════════════════════════════════════════════════╗
║  AI-DRIVEN CABLE DESIGN VALIDATION - WITH HITL SUPPORT               ║
║  Fixed: WARN logic, confidence calibration, HITL for missing data    ║
╚══════════════════════════════════════════════════════════════════════╝


MODE 1: NON-INTERACTIVE VALIDATION (Automated)

🧪 TEST 1: Complete design (no HITL needed)
AI-DRIVEN CABLE DESIGN VALIDATION SYSTEM
Input: Validate IEC 60502-1 cable, 10 sqmm Cu Class 2, PVC insulation 1.0 mm, 0.6/1 kV
Mode: Non-interactive


🎯 SUPERVISOR DECISION: EXTRACT_FROM_TEXT

🔍 EXTRACTED ATTRIBUTES:
   ✓ standard: IEC 60502-1
   ✓ voltage: 0.6/1 kV
   ✓ conductor_material: Cu
   ✓ conductor_class: Class 2
   ✓ csa: 10
   ✓ insulation_material: PVC
   ✓ insulation_thickness: 1.0

✅ ALL ATTRIBUTES PRESENT

🔬 VALIDATION COMPLETE
   Confidence: 1.00
   WARN count: 0, FAIL count: 0, Missing: 0

FINAL VALIDATION RESULTS

✅ standard: PASS
   Expected: IEC 60502-1
   Comment: The specified standa

In [187]:
def demo_non_interactive_mode():
    """Demonstrate non-interactive validation (automated)"""
    print("╔"+"═"*78+"╗")
    print("║" + " "*20 + "NON-INTERACTIVE MODE DEMO" + " "*33 + "║")
    print("╚"+"═"*78+"╝")
    print("\nIn this mode:")
    print("  • Missing fields receive WARN status")
    print("  • No user prompts")
    print("  • Suitable for automated batch validation")
    print("  • Lower confidence scores reflect missing data")

    print("\n" + "─"*80)
    print("TEST 1: Complete design (all fields present)")
    print("─"*80)
    result = run_validation(
        "IEC 60502-1, 0.6/1 kV, Cu Class 2, 10 mm², PVC insulation 1.0 mm",
        verbose=True,
        hitl_mode=False
    )
    print(f"\nResult: {sum(1 for v in result['validation'] if v['status'] == 'PASS')} PASS, "
          f"{sum(1 for v in result['validation'] if v['status'] == 'WARN')} WARN, "
          f"{sum(1 for v in result['validation'] if v['status'] == 'FAIL')} FAIL")

    print("\n" + "─"*80)
    print("TEST 2: Incomplete design (missing standard and voltage)")
    print("─"*80)
    result = run_validation(
        "Cu Class 2, 10 mm², PVC insulation 1.0 mm",
        verbose=True,
        hitl_mode=False
    )
    print(f"\nResult: {sum(1 for v in result['validation'] if v['status'] == 'PASS')} PASS, "
          f"{sum(1 for v in result['validation'] if v['status'] == 'WARN')} WARN, "
          f"{sum(1 for v in result['validation'] if v['status'] == 'FAIL')} FAIL")
    print(f"Notice: Low confidence ({result['confidence']:.2f}) due to missing fields")


def demo_hitl_mode():
    """Demonstrate HITL validation (interactive)"""
    print("\n\n╔"+"═"*78+"╗")
    print("║" + " "*18 + "HUMAN-IN-THE-LOOP MODE DEMO" + " "*32 + "║")
    print("╚"+"═"*78+"╝")
    print("\nIn this mode:")
    print("  • Initial validation shows WARN for missing fields")
    print("  • System prompts user for missing information")
    print("  • Re-validation occurs after data collection")
    print("  • Confidence improves with complete data")

    print("\n" + "─"*80)
    print("TEST: Incomplete design → HITL interaction → Re-validation")
    print("─"*80)
    print("\nInitial input: '10 sqmm copper cable with PVC insulation'")
    print("Missing: standard, voltage, conductor_class, insulation_thickness")
    print("\nThe system will:")
    print("  1. Perform initial validation (WARNs for missing fields)")
    print("  2. Ask if you want to provide missing information")
    print("  3. Collect each missing attribute")
    print("  4. Re-validate with complete data")
    print("  5. Show improved validation results")

    print("\n" + "="*80)
    print("STARTING HITL VALIDATION...")
    print("="*80)

    result = run_validation(
        "10 sqmm copper cable with PVC insulation",
        verbose=True,
        hitl_mode=True
    )

    print("\n" + "="*80)
    print("HITL VALIDATION COMPLETE")
    print("="*80)


def compare_modes():
    """Compare results between non-interactive and HITL modes"""
    print("\n\n╔"+"═"*78+"╗")
    print("║" + " "*25 + "MODE COMPARISON" + " "*38 + "║")
    print("╚"+"═"*78+"╝")

    test_input = "Cu Class 2, 16 mm², PVC insulation 1.0 mm"

    print(f"\nTest input: '{test_input}'")
    print("Missing: standard, voltage")

    print("\n" + "─"*80)
    print("NON-INTERACTIVE MODE:")
    print("─"*80)
    result1 = run_validation(test_input, verbose=False, hitl_mode=False)
    print(f"Status Summary:")
    print(f"  PASS: {sum(1 for v in result1['validation'] if v['status'] == 'PASS')}")
    print(f"  WARN: {sum(1 for v in result1['validation'] if v['status'] == 'WARN')}")
    print(f"  FAIL: {sum(1 for v in result1['validation'] if v['status'] == 'FAIL')}")
    print(f"  Confidence: {result1['confidence']:.2f}")
    print(f"  HITL interactions: {len(result1.get('conversation_history', []))}")

    print("\n" + "─"*80)
    print("HITL MODE (uncomment to test interactively):")
    print("─"*80)
    print("# Uncomment the following line to test HITL mode:")
    print(f"# result2 = run_validation('{test_input}', verbose=True, hitl_mode=True)")
    print("\nExpected behavior:")
    print("  1. Initial validation with 2 WARNs (standard, voltage)")
    print("  2. Prompt: 'Would you like to provide missing information?'")
    print("  3. If yes: Collect standard and voltage")
    print("  4. Re-validate with complete data")
    print("  5. Higher confidence (0.85-0.95)")


def workflow_diagram():
    """Print HITL workflow diagram"""
    print("\n\n╔"+"═"*78+"╗")
    print("║" + " "*28 + "HITL WORKFLOW" + " "*37 + "║")
    print("╚"+"═"*78+"╝")

    diagram = """
    ┌─────────────────────┐
    │   User Input        │
    │ (text or design ID) │
    └──────────┬──────────┘
               │
               ▼
    ┌─────────────────────┐
    │  Extract/Fetch      │
    │  Attributes         │
    └──────────┬──────────┘
               │
               ▼
    ┌─────────────────────┐
    │  Check Missing      │
    │  Attributes         │
    └──────────┬──────────┘
               │
               ▼
    ┌─────────────────────────────────────────┐
    │  INITIAL VALIDATION                     │
    │  • All fields validated                 │
    │  • Missing fields → WARN status         │
    │  • Confidence reflects uncertainty      │
    └──────────┬──────────────────────────────┘
               │
               ▼
         ┌────┴────┐
         │ HITL    │ No → End (validation complete)
         │ Mode?   │
         └────┬────┘
              │ Yes
              ▼
    ┌─────────────────────────────────────────┐
    │  HITL PROMPT                            │
    │  • Show WARN fields                     │
    │  • Ask: "Provide missing info?"         │
    └──────────┬──────────────────────────────┘
               │
         ┌─────┴──────┐
         │ User       │ No → End (keep WARNs)
         │ Responds?  │
         └─────┬──────┘
               │ Yes
               ▼
    ┌─────────────────────┐
    │  Collect Missing    │◄────┐
    │  Attributes         │     │
    │  (one at a time)    │     │ More
    └──────────┬──────────┘     │ missing
               │                │
               └────────────────┘
               │ All collected
               ▼
    ┌─────────────────────────────────────────┐
    │  RE-VALIDATION                          │
    │  • Validate with complete data          │
    │  • Improved confidence                  │
    │  • Fewer/no WARNs                       │
    └──────────┬──────────────────────────────┘
               │
               ▼
    ┌─────────────────────┐
    │  Final Results      │
    │  (with HITL log)    │
    └─────────────────────┘
    """
    print(diagram)

    print("\nKey Benefits of HITL Mode:")
    print("  ✓ Transparent first-pass validation (shows what's missing)")
    print("  ✓ User decides whether to provide additional data")
    print("  ✓ Guided data collection (one field at a time)")
    print("  ✓ Improved validation accuracy after data completion")
    print("  ✓ Audit trail of human interactions")


def usage_examples():
    """Show usage examples"""
    print("\n\n╔"+"═"*78+"╗")
    print("║" + " "*28 + "USAGE EXAMPLES" + " "*35 + "║")
    print("╚"+"═"*78+"╝")

    print("""
# Example 1: Non-interactive validation (automated batch processing)
result = run_validation(
    "IEC 60502-1, 0.6/1 kV, Cu Class 2, 10 mm², PVC 1.0mm",
    hitl_mode=False  # Default, no user interaction
)

# Example 2: HITL validation (interactive mode)
result = run_validation(
    "10 sqmm copper cable",  # Incomplete input
    hitl_mode=True  # Enable human-in-the-loop
)
# System will:
# 1. Validate and show WARNs for missing fields
# 2. Ask: "Would you like to provide missing information?"
# 3. Collect: standard, voltage, class, insulation material, thickness
# 4. Re-validate with complete data

# Example 3: Database design with HITL
result = run_validation(
    "Validate DESIGN-002",  # Has some null fields
    hitl_mode=True
)
# Will prompt for null/missing fields from database

# Example 4: Programmatic access to results
result = run_validation(input_text, verbose=False, hitl_mode=False)
validation_summary = {
    'pass_count': sum(1 for v in result['validation'] if v['status'] == 'PASS'),
    'warn_count': sum(1 for v in result['validation'] if v['status'] == 'WARN'),
    'fail_count': sum(1 for v in result['validation'] if v['status'] == 'FAIL'),
    'confidence': result['confidence'],
    'hitl_interactions': len(result.get('conversation_history', []))
}
""")


if __name__ == "__main__":
    print("""
╔══════════════════════════════════════════════════════════════════════╗
║           HUMAN-IN-THE-LOOP CABLE VALIDATION DEMO                    ║
║                                                                      ║
║  This demo shows two validation modes:                              ║
║  1. Non-interactive: Automated validation with WARNs                ║
║  2. HITL: Interactive data collection and re-validation             ║
╚══════════════════════════════════════════════════════════════════════╝
""")

    # Show workflow
    workflow_diagram()

    # Demo non-interactive mode
    demo_non_interactive_mode()

    # Show mode comparison
    compare_modes()

    # Show usage examples
    usage_examples()

    # Uncomment to test HITL mode interactively
    print("\n" + "="*80)
    print("To test HITL mode interactively, uncomment the following line:")
    print("="*80)
    demo_hitl_mode()

    print("\n✅ Demo complete. See above for HITL workflow and usage examples.")


╔══════════════════════════════════════════════════════════════════════╗
║           HUMAN-IN-THE-LOOP CABLE VALIDATION DEMO                    ║
║                                                                      ║
║  This demo shows two validation modes:                              ║
║  1. Non-interactive: Automated validation with WARNs                ║
║  2. HITL: Interactive data collection and re-validation             ║
╚══════════════════════════════════════════════════════════════════════╝



╔══════════════════════════════════════════════════════════════════════════════╗
║                            HITL WORKFLOW                                     ║
╚══════════════════════════════════════════════════════════════════════════════╝

    ┌─────────────────────┐
    │   User Input        │
    │ (text or design ID) │
    └──────────┬──────────┘
               │
               ▼
    ┌─────────────────────┐
    │  Extract/Fetch      │
    │  Attributes         │
    └─────────